In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow

import torch

print('Cuda ststus: %s' % str(torch.cuda.is_available()))
print('Divices available: %d' % torch.cuda.device_count())
print('Device name: %s' % torch.cuda.get_device_name())

Cuda ststus: True
Divices available: 1
Device name: NVIDIA GeForce GTX 1650


### Spark Session

In [12]:
import findspark
findspark.init('C:\spark')

from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName('Reviews Session')
         .config('spark.executor.memory', '8g')
         .config('spark.executor.cores', '2')
         .config('spark.driver.memory', '4g')
         .getOrCreate())

spark

### Getting unzipped files from other file path

In [10]:
import os
# import zipfile

# def unzip_folders(input_directory, output_directory):
#     # Iterate over each file in the input directory
#     for root, dirs, files in os.walk(input_directory):
#         for file in files:
#             file_path = os.path.join(root, file)

#             # Check if the file is a zip file
#             if file.endswith(".zip"):
#                 # Create a directory to unzip the contents
#                 output_folder = os.path.join(output_directory, os.path.splitext(file)[0])
#                 os.makedirs(output_folder, exist_ok=True)

#                 # Unzip the file
#                 with zipfile.ZipFile(file_path, 'r') as zip_ref:
#                     zip_ref.extractall(output_folder)

# # Example usage:



# input_directory = r"C:\Users\jdieg\Desktop\henry\proyectos\FINAL\data\google_maps"
# output_directory = r"C:\Users\jdieg\Desktop\data_camp\projects\restaurant_reviews\data\google"

# unzip_folders(input_directory, output_directory)


### Have a look at Data Files.
On:
* metadata-sitios (json file)
* reviews (json file)

To get fields names, kind of data, and so on..

#### metadata-sitios (json file)

In [38]:
# Multi Loading json files for metadata
metadata_df = spark.read.json("data/google/*00[2-3]/*/metadata*/*.json")

In [39]:
# Print the Schema to have a loot at structure DataFrame
metadata_df.printSchema()

root
 |-- MISC: struct (nullable = true)
 |    |-- Accessibility: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Activities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Amenities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Atmosphere: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Crowd: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Dining options: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- From the business: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Getting here: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health & safety: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health and safety: array (nullable = true)


In [40]:
# Columns and rows
print(metadata_df.columns)
print('There are %d rows at metadata_df' % metadata_df.count())

['MISC', 'address', 'avg_rating', 'category', 'description', 'gmap_id', 'hours', 'latitude', 'longitude', 'name', 'num_of_reviews', 'price', 'relative_results', 'state', 'url']
There are 3025011 rows at metadata_df


In [42]:
metadata_df.select('MISC').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|MISC                                                                                                                                                                                                                                                                                                                                                                                                        